# Author

**Student Name**:  SHARANYAK PODDER

**Student ID**:  230356331

# Problem formulation

Home vs. Restaurant Food Classification:
   - Problem: Classifying food items prepared  at home or in a restaurant.
   - Model: Binary classification model


**Step_1: Importing Libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

**Step_2:** We install the MLEnd Dataset using pip install function and installing some basic python libraries so that colab can access it.

In [ ]:
!pip install mlend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spkit as sp

from skimage import exposure
from skimage.color import rgb2hsv, rgb2gray
import skimage as ski

import mlend
from mlend import download_yummy

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa

from sklearn.preprocessing import LabelEncoder

drive.mount('/content/drive')

Mounted at /content/drive


**Step_3: Download Data**

Here we will download the complete MLEnd Yummy Dataset, i.e. the MLEnd Yummy Dataset. This dataset consists of a total of 3250 samples from the MLEnd Yummy Dataset corresponding to dishes that contain either rice or chips.

In [ ]:
baseDir = download_yummy(save_to = '/content/drive/MyDrive/Data/MLEnd')
baseDir

100%|▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓|3250\3250|003250.jpg
Done!


'/content/drive/MyDrive/Data/MLEnd/yummy'

In [ ]:
os.listdir(baseDir)

['MLEndYD_images_small',
 'MLEndYD_image_attributes_small.csv',
 'MLEndYD_images',
 'MLEndYD_image_attributes_benchmark.csv']

**Step_4: Understanding Our Dataset**

Each sample in the MLEnd Small Yummy Dataset corresponds to one dish instance and is described by 9 attributes, namely:

Photo of the dish.

Dish name.

Whether home or restaurant.

Cuisine.

Ingredients.

Diet.

Healthiness rating.

Tastiness rating.

Rice or chips?

The CSV file MLEndYD_image_attributes.csv captures the values of all the attributes of each sample. However, instead of an actual photo, this CSV file stores the name of the photo, e.g. '00001.jpg', that is stored in the separate folder MLEndYD_images.

In [ ]:
MLENDYD_df = pd.read_csv('/content/drive/MyDrive/Data/MLEnd/yummy/MLEndYD_image_attributes_benchmark.csv').set_index('filename')
MLENDYD_df

,Diet,Cuisine_org,Cuisine,Dish_name,Home_or_restaurant,Ingredients,Healthiness_rating,Healthiness_rating_int,Likeness,Likeness_int,Benchmark_A
filename,,,,,,,,,,,
000001.jpg,non_vegetarian,japanese,japanese,chicken_katsu_rice,marugame_udon,"rice,chicken_breast,spicy_curry_sauce",neutral,3.0,like,4.0,Train
000002.jpg,non_vegetarian,english,english,english_breakfast,home,"eggs,bacon,hash_brown,tomato,bread,tomato,bake...",unhealthy,2.0,like,4.0,Train
000003.jpg,non_vegetarian,chinese,chinese,spicy_chicken,jinli_flagship_branch,"chili,chicken,peanuts,sihuan_peppercorns,green...",neutral,3.0,strongly_like,5.0,Train
000004.jpg,vegetarian,indian,indian,gulab_jamun,home,"sugar,water,khoya,milk,salt,oil,cardamon,ghee",unhealthy,2.0,strongly_like,5.0,Train
000005.jpg,non_vegetarian,indian,indian,chicken_masala,home,"chicken,lemon,turmeric,garam_masala,coriander_...",healthy,4.0,strongly_like,5.0,Train
...,...,...,...,...,...,...,...,...,...,...,...
003246.jpg,vegetarian,indian,indian,zeera_rice,home,"1_cup_basmati_rice,2_cups_water,2_tablespoons_...",healthy,4.0,strongly_like,5.0,Train
003247.jpg,vegetarian,indian,indian,paneer_and_dal,home,"fried_cottage_cheese,ghee,lentils,milk,wheat_f...",healthy,4.0,strongly_like,5.0,Test
003248.jpg,vegetarian,indian,indian,samosa,home,"potato,onion,peanut,salt,turmeric_powder,red_c...",very_unhealthy,1.0,like,4.0,Test


In [ ]:
sample_path = '/content/drive/MyDrive/Data/MLEnd/yummy/MLEndYD_images/*.jpg'
files = glob.glob(sample_path)
len(files)

3250

**Step_5: Splitting the dataset into Traning and Test Datsets**

In [ ]:
train_data, test_data = train_test_split(MLENDYD_df, test_size=0.2, random_state=42)

# Machine Learning pipeline

Constructs a machine learning pipeline using scikit-learn's Pipeline class.
The pipeline consists of two stages:

1. 'tfidf': Uses TfidfVectorizer to convert the textual descriptions into numerical features using the TF-IDF (Term Frequency-Inverse Document Frequency) representation. It limits the features to 5000 and removes English stop words.

2. 'classifier': Employs a logistic regression classifier (LogisticRegression) to predict the 'preparation_location' based on the TF-IDF features.

**Step_6: Building the Machine Learning Pipeline**

The pipeline used includes text vectorization using TF-IDF and a logistic regression classifier. It allows us to adjust hyperparameters and try different algorithms based on our dataset and requirements.

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('classifier', LogisticRegression())
])

pipeline.fit(train_data['Dish_name'], train_data['Home_or_restaurant'])


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, stop_words='english')),
                ('classifier', LogisticRegression())])

# Transformation stage

We use 'tfidf' which is a user-assigned name for the transformation stage, and TfidfVectorizer is the class responsible for the transformation. TF-IDF stands for Term Frequency-Inverse Document Frequency, a numerical statistic that reflects the importance of a word in a document relative to a collection of documents (corpus).

TfidfVectorizer Parameters:
max_features: Specifies the maximum number of features (unique words or terms) to be considered. In this case, we are setting it to 5000.

stop_words: Specifies that English stop words (common words like "the," "and," etc.)

# Modelling

The two models we used for solving the the problem that wether a dish has been prepared at home or at restaurant are as follows:

**1. TF-IDF Vectorizer:**

The TF-IDF vectorizer is a crucial component of the pipeline. It transforms the raw text data (food descriptions) into numerical features using the TF-IDF algorithm.

**2. Logistic Regression Classifier:**

The logistic regression model is employed for the actual classification task. It predicts whether a food item is prepared at home or in a restaurant based on the features generated by the TF-IDF vectorizer.

**Step_7: Making Predictions and Evaluating the model**

Here, we provide Dish_name as a test data to out ML pipeline and on the basis of that our model evaluates the accuracy and produces a classification report.

In [ ]:
predictions = pipeline.predict(test_data['Dish_name'])

accuracy = accuracy_score(test_data['Home_or_restaurant'], predictions)
report = classification_report(test_data['Home_or_restaurant'], predictions)

print(f'Accuracy: {accuracy}')
print('\nClassification Report:\n', report)


Accuracy: 0.6846153846153846

Classification Report:
                                  precision    recall  f1-score   support

                 36_street_food       0.00      0.00      0.00         1
         albura_kathisma_bistro       0.00      0.00      0.00         1
            anjappar_restaurant       0.00      0.00      0.00         1
                           asda       0.00      0.00      0.00         2
                 buns_&_berries       0.00      0.00      0.00         1
              burgitzza_watford       0.00      0.00      0.00         1
                   chicken_shop       0.00      0.00      0.00         2
                   coco_di_mama       0.00      0.00      0.00         1
                 dartmoor_lodge       0.00      0.00      0.00         1
                  delamina_east       0.00      0.00      0.00         2
                      eat_tokyo       0.00      0.00      0.00         1
                efes_restaurant       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


# Methodology

**Training and Validation Steps:**

Training and validating machine learning models involves several steps, including data splitting, model training, and performance assessment. In the context of the provided machine learning pipeline for classifying food items.

**Model Performance Assessment:**

1. Accuracy

2. Classification Report

# Dataset

We used the MLend_yummy_dataset to create and validate our model.

**Step_8: Testing the model**

Here we test our model by providing it with a dish name such as chicken_masala from the Mile End Yummy Dataset and our model will predict wether its a dish prepared at home or restaurant.

In [ ]:
# Example prediction for a new food description
new_food_description = ["chicken_masala"]
new_prediction = pipeline.predict(new_food_description)
print(f'Prediction for new data: {new_prediction}')

Prediction for new data: ['home']


# Result

Here we tried providing our model with a random dish from the dataset mentioned above and as expected our model made the right prediction by telling that chicken_masala was a dish prepared at home.


# Conclusion

From the above problem and the solution to build the machine learning pipeline to predict wether a dish is prepared at home or at restaurant we can conclude that the pipeline is working properly and is providing us with a 68% accuracy.